### Paso 1: Importación de variables

In [ ]:
import pandas as pd
import time
import random
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle

### Paso 2: Definición de opciones para el navegador

In [ ]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

### Paso 3: Estructuración del WebScraping para captura de links

In [ ]:
# PASO 3.1: Definición de variables para captura de información -----------------------------------------------------------------------#
headers = []
headers_text = []
headers_link = []
all_news = []

# PASO 3.1.1 Definición de frases de interés a consultar (serviran para iteración en ciclo) -------------------------------------------#
phrases_of_interest = [
    'Cambio climático',
    'Recalentamiento global',
    'Emisiones de CO2',
    'Gases de efecto invernadero (GEI)',
    'Temperatura'
    'Oleada de calor',
    'Muerte por calor',
    'Contaminación del aire',
    'Calidad del aire',
    'Polución',
    'Descontaminación',
    'Contaminación del agua',
    'Acidificación del océano',
    'Desastre ambiental',
    'Catástrofe ambiental',
    'Deforestación',
    'Reforestación',
    'Tala de árboles',
    'Siembra de árboles',
    'Reciclaje',
    'Economía circular',
    'Matriz energética',
    'Energía limpia',
    'Energía renovable',
    'Energías alternativas',
    'Energías verdes',
    'Derecho ambiental',
    'Regulación ambiental',
    'Normativa ambiental',
    'Convención Marco de las Naciones Unidas sobre Cambio Climático,',
    'Acuerdo de París',
    'Protocolo de Tokio',
    'Transición energética',
    'Sostenibilidad ambiental',
    'Refugiado climático ',
    'Estrategia 2050'
]

In [ ]:
# PASO 3.2: Aviso para el usuario final de que se inicio el web-scraping --------------------------------------------------------------#
print("Se inició el Web Scraping a 'Semana', por favor espere...") # Mensaje para visualización del inicio del proceso

In [ ]:
# PASO 3.3: Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
service = Service(executable_path=r'../chromedriver-win64/chromedriver.exe') # Creación del servicio mediante el driver
options = webdriver.ChromeOptions() # Definición de variable con las opciones para chrome
driver = webdriver.Chrome(service=service, options=options) # Creación del objeto del WebDriver
driver.get("https://www.semana.com/") # Página principal
time.sleep(random.randrange(1,5)) # Simulación de comportamiento humano

In [ ]:
# PASO 3.4: Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
button_xpath = '//*[@id="fusion-app"]/div/header/div[3]/div[4]/div/div[1]/div[1]/div[1]/div[1]/div/label' # xpath del botón
menu_button = driver.find_element(By.XPATH, button_xpath) # Botón de lupa (sidebar)
menu_button.click(); # Acción de click en el botón
time.sleep(random.randrange(1,5)) # Simulación de comportamiento humano

In [ ]:
# PASO 3.5: Ubicación y selección del campo para búsqueda -----------------------------------------------------------------------------#
search_bar = driver.find_element(By.XPATH, '//*[@id="queryly_query"]') # Campo de búsqueda (ventana emergente)

In [ ]:
# PASO 3.6: Ciclo para iterar sobre las frases de interés en el campo para búsqueda ---------------------------------------------------#
for phrases in phrases_of_interest:
    search_bar.clear() # Limpieza del campo
    search_bar.send_keys(phrases) # Envío de la frase de interés al campo de búsqueda
    time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano
    search_bar.send_keys(Keys.RETURN) # Envío de consulta (Enter)    
    
    # 3.6.1 Control de excepción en caso dado que el filtro de "Medio ambiente" no esté presente en la página
    try:
        # Si está presente el filtro se aplica
        css_selector = 'div.filter_item[data-filter-value="Medio ambiente"] a' # Definición del css del filtro
        filter_button = driver.find_element(By.CSS_SELECTOR, css_selector) # Intento de encontrar el botón para aplicar el filtro
        filter_button.click() # Acción de click en el botón
    except NoSuchElementException:
        pass # Si NO está presente el filtro NO se aplica y la iteración continua
    
    
    # 
    # Código que captura los links
    #

    # 3.6.x Repetición del proceso de búsqueda
    button_xpath = '//*[@id="fusion-app"]/div/header/div[2]/div[3]/div/div[1]/div[1]/div[1]/div[1]/div/label' # xpath del botón
    menu_button = driver.find_element(By.XPATH, button_xpath) # Botón de lupa (sidebar)
    menu_button.click(); # Acción de click en el botón
    time.sleep(random.randrange(1,5)) # Simulación de comportamiento humano
    search_bar = driver.find_element(By.XPATH, '//*[@id="queryly_query"]') # Campo de búsqueda (ventana emergente)
    
    time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano


In [ ]:
driver.quit()